The aim of this notebook is to generate a csv file with the results of the metrics applied on synthethic drifted data.

In [16]:
import sys, os
sys.path.append(os.path.abspath('Utils'))
sys.path.append(os.path.abspath('data'))
sys.path.append(os.path.abspath('thresholds_and_results'))

from utils_generateTests import split_data, reduced_on_drift_kdim, test_on_reduced_kdim
from utils_dimRedDef import find_dimensions_number, initialize_DimReduction, scale_dataset, init_scaler
from utils_resNet import df_from_folder, init_resnet

In [17]:
# params
seed_split = 2
# seed_split = 1
seed_drift = 10
seed_metrics = 100
info_dataset = [seed_split, seed_drift, seed_metrics]


In [18]:
# number of dimensions for dimensionality reduction
k = 6
# csv file where saving results
resultFile = 'thresholds_and_results/2dim/devResults_2d.csv'


In [19]:
# initialize the resNet, used as pre-process for each image in order to extract more relevant feature and not work directly with pixels
model = init_resnet(seed_split)

c:\Users\xausa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\xausa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Defining folders

In [20]:
# defining folder directories (created in drift_and_thresholds.ipynb)
intensity_path = 'data/synthetic_data/drift_intensity/'
gaussian_path_1 = 'data/synthetic_data/drift_gaussian_1/'
gaussian_path_10 = 'data/synthetic_data/drift_gaussian_10/'
gaussian_path_100 = 'data/synthetic_data/drift_gaussian_100/'
input_path = 'data/synthetic_data/black/'

In [21]:
# split dataframe in train, test, validation sets, as lists
train_list, test_list, val_list = split_data(input_path, seed_split)

In [22]:
# create dataframes needed as input for the governance pipeline 
train = df_from_folder(input_path, model,  train_list)
test = df_from_folder(input_path,  model,  test_list)
val = df_from_folder(input_path,  model, val_list)
val_intensity = df_from_folder(intensity_path, model, val_list)
val_gaussian_1 = df_from_folder(gaussian_path_1, model,  val_list)
val_gaussian_10 = df_from_folder(gaussian_path_10,  model, val_list)
val_gaussian_100 = df_from_folder(gaussian_path_100,  model, val_list)

KeyboardInterrupt: 

Find PCA with components that get 80% of variance on test set as done in the paper 'Failing Loudly' by Rabanser & co. to find the number of components to reduce the dataframes to. 

In [ ]:
# k = find_dimensions_number(test)      
k=6

Initialize dimensionality reductors

In [ ]:
test

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.178436,0.110992,1.646519,0.461198,0.576611,0.440900,0.378514,0.234400,1.356820,1.447873,...,0.412051,1.002263,0.062367,0.291720,0.514440,0.285739,0.545336,0.095519,0.406564,0.453166
1,0.157798,0.182142,1.353120,0.302760,0.396004,0.293400,0.443676,0.235815,1.362739,1.114463,...,0.748258,0.325900,0.073889,0.395346,0.848227,0.415397,0.433709,0.223282,0.281199,0.227835
2,0.279244,0.173864,1.204811,0.534460,0.455656,0.253122,0.358214,0.369609,1.148496,0.855464,...,0.608319,0.399806,0.100406,0.550980,1.274041,0.404865,0.284395,0.353530,0.579028,0.341356
3,0.141640,0.115692,1.701929,0.543042,0.615419,0.434615,0.388934,0.364442,1.280295,1.498631,...,0.356081,1.112890,0.094736,0.261186,0.535325,0.439572,0.627597,0.107170,0.455620,0.319513
4,0.194263,0.224569,1.435983,0.387076,0.667164,1.001340,0.635107,0.232524,1.038372,0.860122,...,0.773579,0.484655,0.131175,0.604059,1.274567,0.444980,0.838310,0.115054,0.649787,0.242541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.154442,0.177273,1.153289,0.312910,0.452722,0.187168,0.375944,0.239255,1.366748,0.920817,...,0.796336,0.431720,0.063977,0.343481,0.856915,0.324841,0.377139,0.310673,0.318942,0.454552
146,0.129999,0.200820,1.077242,0.186530,0.494382,0.203442,0.653930,0.407251,1.314530,0.551926,...,0.795509,0.336958,0.080450,0.591140,0.980752,0.610761,0.392429,0.279317,0.414537,0.247657
147,0.174388,0.154209,1.678751,0.507721,0.661812,0.469164,0.516615,0.286612,1.498291,1.405413,...,0.439443,1.039569,0.051056,0.350483,0.724283,0.405356,0.662700,0.152810,0.507256,0.486962
148,0.136932,0.155063,1.461972,0.203437,0.428063,0.150663,0.630323,0.451971,1.328927,1.110436,...,0.624931,0.462982,0.052585,0.283102,1.039518,0.395361,0.453809,0.170750,0.317637,0.337664


In [ ]:
# initialize scaler for autoencoder
standard_scaler = init_scaler(train)


In [ ]:
# scaling source dataset
train_scaled = scale_dataset(train, standard_scaler)
test_scaled = scale_dataset(test, standard_scaler)
val_scaled = scale_dataset(val, standard_scaler)
# scaling drifted dataset
val_intensity_scaled = scale_dataset(val_intensity, standard_scaler)
val_gaussian_1_scaled = scale_dataset(val_gaussian_1, standard_scaler)
val_gaussian_10_scaled = scale_dataset(val_gaussian_10, standard_scaler)
val_gaussian_100_scaled = scale_dataset(val_gaussian_100, standard_scaler)

In [ ]:
# initialize dimensionality reduction
reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer = initialize_DimReduction(seed_metrics, test,  train_scaled, test_scaled, k=k)   #k not specified: k=2

c:\Users\xausa\AppData\Local\Programs\Python\Python310\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")




Epoch 1/15


7/7 [==============================] - 2s 53ms/step - loss: 0.3578 - mae: 0.3578 - val_loss: 0.3115 - val_mae: 0.3115
Epoch 2/15
7/7 [==============================] - 0s 11ms/step - loss: 0.2670 - mae: 0.2670 - val_loss: 0.2135 - val_mae: 0.2135
Epoch 3/15
7/7 [==============================] - 0s 11ms/step - loss: 0.1864 - mae: 0.1864 - val_loss: 0.1677 - val_mae: 0.1677
Epoch 4/15
7/7 [==============================] - 0s 11ms/step - loss: 0.1554 - mae: 0.1554 - val_loss: 0.1526 - val_mae: 0.1526
Epoch 5/15
7/7 [==============================] - 0s 8ms/step - loss: 0.1447 - mae: 0.1447 - val_loss: 0.1470 - val_mae: 0.1470
Epoch 6/15
7/7 [==============================] - 0s 8ms/step - loss: 0.1403 - mae: 0.1403 - val_loss: 0.1436 - val_mae: 0.1436
Epoch 7/15
7/7 [==============================] - 0s 8ms/step - loss: 0.1366 - mae: 0.1366 - val_loss: 0.1391 - val_mae: 0.1391
Epoch 8/15
7/7 [==============================] - 0s 9ms/step - loss: 0.1304 - mae: 0.1304 - val

In [ ]:
# Apply dimensionality reduction
test_dim_red, info_drift = reduced_on_drift_kdim(test,  info_dataset,  reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer, )
val_dim_red, info_drift_val= reduced_on_drift_kdim(val,  info_dataset,  reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer,)
#  on drifted
gau1_dim_red, info_drift_g1 = reduced_on_drift_kdim(val_gaussian_1, info_dataset, reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer ,  sigma=1, drift='Gaussian')
gau10_dim_red, info_drift_g10 = reduced_on_drift_kdim(val_gaussian_10, info_dataset, reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer,   sigma=10, drift='Gaussian')
gau100_dim_red, info_drift_g100 = reduced_on_drift_kdim(val_gaussian_100, info_dataset, reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer,   sigma=100, drift='Gaussian')
intensity_dim_red, info_drift_i = reduced_on_drift_kdim(val_intensity, info_dataset, reducer_pca, reducer_umap, U_encoder_layer, T_encoder_layer ,  sigma=42, drift='intensity')


5/5 [==============================] - 0s 0s/step


Apply tests

In [ ]:
test_on_reduced_kdim(val, test,  val_dim_red, test_dim_red, seed_metrics, resultFile, info_drift = info_drift_val, k=k)
test_on_reduced_kdim(val_gaussian_1, test, gau1_dim_red, test_dim_red,  seed_metrics, resultFile, info_drift_g1, k)
test_on_reduced_kdim(val_gaussian_10, test,  gau10_dim_red, test_dim_red, seed_metrics, resultFile, info_drift_g10, k)
test_on_reduced_kdim(val_gaussian_100, test, gau100_dim_red, test_dim_red,  seed_metrics, resultFile, info_drift_g100, k)
test_on_reduced_kdim(val_intensity, test, intensity_dim_red, test_dim_red, seed_metrics, resultFile, info_drift_i, k)
